# $\rho'$ analysis notes

## GIRD Selection criteria:

These criteria were applied on the selection stage(GRID):

Events:

- \>= 4 tracks
    
Tracks:

- Has Point On inner OR outer ITS Layer
- Not ITS SA
- |dca1| < 3 && |dca0| < 3;

### Data info

In [1]:
from modules.FourTrackEvents import *
from modules.physics import kinematics
from modules.physics.analysis.ITSvsTPC_events import *
from modules.physics.analysis.crossection import GetCrossSection
from importlib import reload

%matplotlib widget
if os_name == 'posix':
    %jsroot on


In [2]:
plt.style.use(hep.style.ROOT)

ft = FourTrackEvents()

print('Events attributes:')
print(ft.orig_events.columns.tolist())
print('Track attributes:')
print(ft.orig_tracks.columns.tolist())
print()
print('Events count:', "{:,}".format(len(ft.orig_events.index)))
print('Tracks count:', "{:,}".format(len(ft.orig_tracks.index)))

Events attributes:
['RunNum', 'PeriodNumber', 'OrbitNumber', 'BunchCrossNumber', 'Mass', 'Pt', 'Q', 'Rapidity', 'Phi', 'ZNAenergy', 'ZNCenergy', 'ZPAenergy', 'ZPCenergy', 'VtxX', 'VtxY', 'VtxZ', 'VtxContrib', 'VtxChi2', 'VtxNDF', 'SpdVtxX', 'SpdVtxY', 'SpdVtxZ', 'SpdVtxContrib', 'V0Adecision', 'V0Cdecision', 'ADAdecision', 'ADCdecision', 'V0Afired', 'V0Cfired', 'ADAfired', 'ADCfired', 'STPfired', 'SMBfired', 'SM2fired', 'SH1fired', 'OM2fired', 'OMUfired', 'IsTriggered', 'nTracklets', 'nTracks', 'ZDCAtime_0', 'ZDCAtime_1', 'ZDCAtime_2', 'ZDCAtime_3', 'ZDCCtime_0', 'ZDCCtime_1', 'ZDCCtime_2', 'ZDCCtime_3', 'FORChip']
Track attributes:
['T_NumberOfSigmaTPCPion', 'T_Eta', 'T_Phi', 'T_Px', 'T_Py', 'T_Pz', 'T_Q', 'T_HasPointOnITSLayer0', 'T_HasPointOnITSLayer1', 'T_ITSModuleInner', 'T_ITSModuleOuter', 'T_TPCNCls', 'T_TPCRefit']

Events count: 110,004
Tracks count: 26,978,093


## Analysis criteria

Let's take from input date only events with 4 tracks and zero total charge.
Also prepare mask for tracks that satisfy standatd criteria contains further conditions for the TPC:

* |NumberOfSigmaTPCPion| < 3
* Number of TPC Clusters > 50
* TPCRefit
## Low energy tracks and TPC

There is an idea about that tracks with small energies (low pt) not able to reach TPC.
We can try to estimate influence of TPC criteria to the statistics.

Let's see what happend in case of we will plot pt for any combination of TPC tracks:

### Event with std criteria

|detector|track1|track2|track3|track4|
|:--:|:--:|:--:|:--:|:--:|
|ITS|+|+|+|+|
|TPC|+|+|+|+|

### Let's start to take also such combinations to the sample:

|detector|track1|track2|track3|track4|
|:--:|:--:|:--:|:--:|:--:|
|ITS|+|+|+|+|
|TPC|+|+|+|-|

|detector|track1|track2|track3|track4|
|:--:|:--:|:--:|:--:|:--:|
|ITS|+|+|+|+|
|TPC|+|+|-|-|

|detector|track1|track2|track3|track4|
|:--:|:--:|:--:|:--:|:--:|
|ITS|+|+|+|+|
|TPC|+|-|-|-|

|detector|track1|track2|track3|track4|
|:--:|:--:|:--:|:--:|:--:|
|ITS|+|+|+|+|
|TPC|-|-|-|-|

We can see statistics gain ration equal 2.5 between std criteria and total combinations:


In [3]:
# ft - four tracks
# zq - zero charge
# nzq - non zero charge
# Pt - transversal momentum
# nTPc - each index(i) of list corresponding to case when more than i TPC tracks in event 
# e.g. ft_nzq_Pt_nTpc contains list of the list of four track events with non zero total charge
# where index (i) of the main list corresponding to case when more than i TPC tracks in event 

ft_zq_Pt_nTpc = []
ft_nzq_Pt_nTpc = []
ft_zq_Mass_nTpc = []
ft_nzq_Mass_nTpc = []
for i in range(5):
    ft_zq_Pt_nTpc.append(kinematics.pt_events(ft.ft_zq_nTPC[i]))
    ft_nzq_Pt_nTpc.append(kinematics.pt_events(ft.ft_nzq_nTPC[i]))
    ft_zq_Mass_nTpc.append(kinematics.mass_events(ft.GetTracksWithPtLt(ft.ft_zq_nTPC[i])))
    ft_nzq_Mass_nTpc.append(kinematics.mass_events(ft.GetTracksWithPtLt(ft.ft_nzq_nTPC[i])))

## False triggering

There are some situations when CCUP9 trigger could be fired false:
It may occured when some fake or random track fires FOR and trigger will provide.

We can check list of FORs of event and what chipkey has each of four tracks.
In case it has matches and produce back to back topology this means correct trigger state.

![img](https://sun9-58.userapi.com/impf/x7UtIW5ElLKpDl4ASPuz0FXhNjwnxYcAy0BuHw/wJZr1On9l4o.jpg?size=1280x718&quality=96&sign=1ed3d5f08fcdefd89ab4e02a5041c6d0&type=album)

See debugging details in [one of the issue](https://github.com/bdrum/cern-physics/issues/42)


In [4]:
# let's form fired FORs numbers for 4 tracks zq EVENT 
for_sensors = pd.DataFrame(ft.orig_events.FORChip.loc[ft.four_tracks_zq.reset_index().entry])
for_sensors['subentry']=for_sensors.groupby(level=0).cumcount()
for_sensors.index.name = 'entry'
for_sensors.set_index('subentry', append=True,inplace=True)

# form arrays of numbers of sensor that linked with TRACKS 
df = ft.four_tracks_zq[['T_ITSModuleInner','T_ITSModuleOuter']] // 1000000

# combine into one dataframe
df['T_ITSSensorNum'] = for_sensors
df = df.reset_index()

# check that sensor linked with track will provide trigger (match with FOR sensor)
df["Inner_matched"] = (abs(abs(df.T_ITSModuleInner*5 - df.T_ITSSensorNum) - 2).apply(lambda x: x <= 2)) * df.T_ITSSensorNum.apply(lambda x: x < 400)

df["Outer_matched"] = (abs(abs(df.T_ITSModuleOuter*5 - df.T_ITSSensorNum) - 2).apply(lambda x: x <= 2)) * df.T_ITSSensorNum.apply(lambda x: x >= 400)
df["vPhiInner"] = np.nan
df.vPhiInner = df.vPhiInner.astype('object')
df["vPhiOuter"] = np.nan
df.vPhiOuter = df.vPhiOuter.astype('object')

df["vPhiInnerValue"] = np.nan
df.vPhiInnerValue = df.vPhiInnerValue.astype('object')
df["vPhiOuterValue"] = np.nan
df.vPhiOuterValue = df.vPhiOuterValue.astype('object')

for i in tqdm(range(len(df))):
    vPhiInner1 = np.zeros(20, dtype=bool)
    vPhiOuter1 = np.zeros(40, dtype=bool)

    vPhiInnerValues = pd.unique(df.T_ITSSensorNum[i][df.Inner_matched[i]]//20)
    vPhiOuterValues = pd.unique((df.T_ITSSensorNum[i][df.Outer_matched[i]]-400)//20)

    vPhiInner1[vPhiInnerValues]=True
    df.vPhiInner[i] = vPhiInner1

    vPhiOuter1[vPhiOuterValues]=True
    df.vPhiOuter[i] = vPhiOuter1

    df.vPhiInnerValue[i] = vPhiInnerValues
    df.vPhiOuterValue[i] = vPhiOuterValues

df_dbg = df.copy()

# take only matched tracks and fill vPhi arrays for inner and outer
# df = df[(df.Inner_matched.apply(any) | df.Outer_matched.apply(any))][['entry', 'vPhiInner', 'vPhiOuter']].groupby('entry').sum()
df = df[['entry', 'vPhiInner', 'vPhiOuter']].groupby('entry').sum()

df["triggered"] = False

# check incorrect topology 

for t in tqdm(df.index):
    for i in range(10):
        for j in range(2):
            k = 2*i+j
            if  (df.vPhiOuter[t][k]    or df.vPhiOuter[t][k+1]        or df.vPhiOuter[t][k+2]) \
		    and (df.vPhiOuter[t][k+20] or df.vPhiOuter[t][(k+21)%40]  or df.vPhiOuter[t][(k+22)%40]) \
		    and (df.vPhiInner[t][i]    or df.vPhiInner[t][i+1]) \
		    and (df.vPhiInner[t][i+10] or df.vPhiInner[t][(i+11)%20]):
                df.at[t,'triggered'] = True

triggered_events = df.index[df.triggered]
untriggered_events = df.index[~df.triggered]

ft_zq_Pt_nTpc_triggered = []
ft_zq_Pt_nTpc_UNtriggered = []

ft_nzq_Pt_nTpc_triggered = []
ft_nzq_Pt_nTpc_UNtriggered = []

for i in range(5):
  ft_zq_Pt_nTpc_triggered.append(ft_zq_Pt_nTpc[i][np.intersect1d(ft_zq_Pt_nTpc[i].index.values, triggered_events.values)])
  ft_zq_Pt_nTpc_UNtriggered.append(ft_zq_Pt_nTpc[i][np.intersect1d(ft_zq_Pt_nTpc[i].index.values, untriggered_events.values)])


  0%|          | 0/77824 [00:00<?, ?it/s]<ipython-input-4-05f57b8ade7d>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.vPhiInner[i] = vPhiInner1
<ipython-input-4-05f57b8ade7d>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.vPhiOuter[i] = vPhiOuter1
<ipython-input-4-05f57b8ade7d>:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.vPhiInnerValue[i] = vPhiInnerValues
<ipython-input-4-05f57b8ade7d>:42: SettingWithCopyWarning:

### Trigger stats

In [5]:
trig_ratio_dict = {}
i = 0
trig_ratio_dict['nTPC'] = list(range(5))
trig_ratio_dict['total_cnt'] = [len(x) for x in ft_zq_Pt_nTpc]
trig_ratio_dict['trig_cnt'] = [len(x) for x in ft_zq_Pt_nTpc_triggered]
trig_ratio_dict['untrig_cnt'] = [len(x) for x in ft_zq_Pt_nTpc_UNtriggered]

rts = pd.DataFrame.from_dict(trig_ratio_dict) 
rts['trig % from ttl'] = (100 * rts.trig_cnt / rts.total_cnt).round(decimals=1)
rts['untrig % from ttl'] = (100 * rts.untrig_cnt / rts.total_cnt).round(decimals=1)
rts

,nTPC,total_cnt,trig_cnt,untrig_cnt,trig % from ttl,untrig % from ttl
0,0,19456,10426,9030,53.6,46.4
1,1,17864,10209,7655,57.1,42.9
2,2,16698,9637,7061,57.7,42.3
3,3,13101,7632,5469,58.3,41.7
4,4,5991,3536,2455,59.0,41.0


## Transversal momentum distribution

We can see what happend in case of we will make criteria for TPC softer, i.e. not each of four tracks
should be TPC refitted, but only 3 etc.

In [6]:
colors = ['red', 'yellow', 'green', 'orange', 'black']
labels = ['ITS & (>= 0TPC)', 'ITS & (>= 1TPC)',
          'ITS & (>= 2TPC)', 'ITS & (>= 3TPC)', 'ITS & ( =  4TPC)']

fig = ShowComparisonSame('', ft_zq_Pt_nTpc, r'$p_t \pi^+\pi^-\pi^+\pi^-$,GeV',
                   labels, colors, nBins=100, ranges=(0, 2),showPatch=True)

# plt.show()
# fig


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Let' see what exactly gain we have in the same order:

0TPC - 1TPC

...

In [7]:
%matplotlib widget

fig, ax = plt.subplots(2,2)
fig.suptitle(r'$p_t$ of 4 track events')
n = [-1,1,0,1,0]
a = [-1,1,1,0,0]
lbls = ['nTPC=3', 'nTPC=2', 'nTPC=1', 'nTPC=0', '']
for i in range(4,0,-1):
    _ = ax[a[i],n[i]].hist(ft_zq_Pt_nTpc[i-1][ft_zq_Pt_nTpc[i-1].index.difference(ft_zq_Pt_nTpc[i].index)], bins=100, range=(0, 2), histtype='step', color='black', linewidth=1.2, label=lbls[::-1][i])
    ax[a[i],n[i]].legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
%matplotlib widget

fig, ax = plt.subplots(1,3, figsize=(20,7))
fig.suptitle(r'$p_t$ of 4 track events')
lbls = ['nTPC=3', 'nTPC=2', 'nTPC=1', 'nTPC=0', '']
i=1
_ = ax[0].hist(ft_zq_Pt_nTpc[i], bins=100, range=(0, 2), histtype='step', color='black', linewidth=1.2, label=f'nTPC>={i}')
_ = ax[1].hist(ft_zq_Pt_nTpc[i-1][ft_zq_Pt_nTpc[i-1].index.difference(ft_zq_Pt_nTpc[i].index)], bins=100, range=(0, 2), histtype='step', color='black', linewidth=1.2, label=lbls[::-1][i])
_ = ax[2].hist(ft_zq_Pt_nTpc[i-1], bins=100, range=(0, 2), histtype='step', color='black', linewidth=1.2, label=f'nTPC>={i-1}')
ax[0].legend()
ax[1].legend()
ax[2].legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
%matplotlib widget

plt.style.use(hep.style.ROOT)
fig = plt.figure()
ax = fig.add_subplot()

fig.suptitle(f'$p_t$ of events')
b = 100
r = 0,2
counts,bin_edges = np.histogram(ft_zq_Pt_nTpc[2], bins=b, range=r)

bin_centres = (bin_edges[:-1] + bin_edges[1:])/2.
errs = np.sqrt(counts)
ax.errorbar(bin_centres, counts, yerr=errs, fmt='.', color='green')

val=(r[1]-r[0])*1000 // b
ax.set_xlabel('$p_t, GeV$')
ax.set_ylabel(f'#events / {val}MeV')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 1, '#events / 20MeV')

Now we can try to estimate what influence has [incorrectness triggering](#False-triggering)

In [621]:
plt.style.use(hep.style.ROOT)
fig = plt.figure()
ax = fig.add_subplot()

fig.suptitle(f'$p_t$ of events')
b = 100
r = 0,2
counts,bin_edges = np.histogram(ft_zq_Pt_nTpc_triggered[2][ind.index][~ind], bins=b, range=r)
# counts,bin_edges = np.histogram(kinematics.pt_events(ft.GetTracksWithNTPC(ft.four_tracks_zq)), bins=b, range=r)

bin_centres = (bin_edges[:-1] + bin_edges[1:])/2.
errs = np.sqrt(counts)
ax.errorbar(bin_centres, counts, yerr=errs, fmt='.', color='green', label=r'after zdc cuts')

val=(r[1]-r[0])*1000 // b
ax.set_xlabel('$p_t, GeV$')
ax.set_ylabel(f'#events / {val}MeV')
# ax.add_patch(Rectangle((0, 0.15), 0.15, 700, fc='lightgrey', alpha=0.4))
# ax.text(0.15,0,"0.15", size=20)
# counts,bin_edges = np.histogram(ft_zq_Pt_nTpc_UNtriggered[2], bins=b, range=r)
counts,bin_edges = np.histogram(ft_zq_Pt_nTpc_triggered[2], bins=b, range=r)
# counts,bin_edges = np.histogram(kinematics.pt_events(ft.GetTracksWithNTPC(ft.four_tracks_zq)), bins=b, range=r)
bin_centres = (bin_edges[:-1] + bin_edges[1:])/2.
errs = np.sqrt(counts)
# ax.errorbar(bin_centres, counts, yerr=errs, fmt='.', label=r'orig plot', color='red')
ax.legend()
# fig


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [622]:
%matplotlib widget

fig, ax = plt.subplots(1,3, figsize=(20,7))
fig.suptitle(r'zdc cut for $p_t$ of 4 track events')
lbls = ['nTPC=3', 'nTPC=2', 'nTPC=1', 'nTPC=0', '']
i=1

b = 100
r = 0,2
counts,bin_edges = np.histogram(ft_zq_Pt_nTpc_triggered[2], bins=b, range=r)
# counts,bin_edges = np.histogram(kinematics.pt_events(ft.GetTracksWithNTPC(ft.four_tracks_zq)), bins=b, range=r)

bin_centres = (bin_edges[:-1] + bin_edges[1:])/2.
errs = np.sqrt(counts)
ax[0].errorbar(bin_centres, counts, yerr=errs, fmt='.', color='green', label=r'before zdc cut')


counts,bin_edges = np.histogram(ft_zq_Pt_nTpc_triggered[2][ind.index][ind], bins=b, range=r)
bin_centres = (bin_edges[:-1] + bin_edges[1:])/2.
errs = np.sqrt(counts)

ax[1].errorbar(bin_centres, counts, yerr=errs, fmt='.', color='black', label=r'zdc cut values')


counts,bin_edges = np.histogram(ft_zq_Pt_nTpc_triggered[2], bins=b, range=r)
bin_centres = (bin_edges[:-1] + bin_edges[1:])/2.
errs = np.sqrt(counts)

ax[2].errorbar(bin_centres, counts, yerr=errs, fmt='.', color='red', label=r'before zdc cut')

counts,bin_edges = np.histogram(ft_zq_Pt_nTpc_triggered[2][ind.index][~ind], bins=b, range=r)
# counts,bin_edges = np.histogram(kinematics.pt_events(ft.GetTracksWithNTPC(ft.four_tracks_zq)), bins=b, range=r)

bin_centres = (bin_edges[:-1] + bin_edges[1:])/2.
errs = np.sqrt(counts)
ax[2].errorbar(bin_centres, counts, yerr=errs, fmt='.', color='green', label=r'after zdc cut')

ax[0].legend()
ax[1].legend()
ax[2].legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Now  let's try to see what tracks we lost from signal area and what contribution they have:

TPC and ITS has different coverage for polar angle:

![img1](https://camo.githubusercontent.com/9a7ab40d0f74a866a7095986644134a0f72cc58b/68747470733a2f2f70702e757365726170692e636f6d2f633835323231362f763835323231363738332f3131396137332f304f76685f6c544b4e7a552e6a7067)

Perhaps we have tracks that not only can't reach TPC, but also has $\theta$ values that TPC doesn't cover.

Below we can see polar angle distribution for tracks that covers three cases:

1. All tracks from events were reconstructed by ITS and TPC
2. Only ITS tracks from events with only part TPC tracks. Here tracks that not reconstructed by TPC
3. All tracks from events were reconstructed by ITS or TPC

We can see small gaps with for the second case, that allow to speak about correctness of the suggestion, but anyway low energy of tracks is the main reason why TPC can't reconstructed tracks.  

## Mass
Let's see on the mass distribution of the events


In [11]:
fig = ShowComparisonSame('4track events Mass', ft_zq_Mass_nTpc, '$Mass, GeV$',
                   labels, colors, nBins=100, ranges=(0.5, 3.5))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Let's compare masses of events with handle triggering manipulations:

_see False triggering chapter below_

In [21]:
plt.style.use(hep.style.ROOT)
fig = plt.figure() #(figsize=(15,7))
ax = fig.add_subplot()

ft_zq_Mass_nTpc_triggered = []
ft_zq_Mass_nTpc_UNtriggered = []

for i in range(5):
  ft_zq_Mass_nTpc_triggered.append(ft_zq_Mass_nTpc[i][np.intersect1d(ft_zq_Mass_nTpc[i].index.values, triggered_events.values)])
  ft_zq_Mass_nTpc_UNtriggered.append(ft_zq_Mass_nTpc[i][np.intersect1d(ft_zq_Mass_nTpc[i].index.values, untriggered_events.values)])


fig.suptitle(f'4 tracks mass', fontsize=32)
b = 25
r = 0.5,3
nTPC = 2
counts,bin_edges = np.histogram(ft_zq_Mass_nTpc_triggered[nTPC], bins=b, range=r)
# counts,bin_edges = np.histogram(ft_zq_Mass_nTpc_triggered[nTPC][ind.index][~ind], bins=b, range=r)
# counts,bin_edges = np.histogram(ft_zq_Mass_nTpc[nTPC], bins=b, range=r)
bin_centres = (bin_edges[:-1] + bin_edges[1:])/2.
errs = np.sqrt(counts)
ax.errorbar(bin_centres, counts, yerr=errs, fmt='.', color='green')#, label=r"after all cuts")

# counts,bin_edges = np.histogram(ft_zq_Mass_nTpc_UNtriggered[nTPC], bins=b, range=r)
counts,bin_edges = np.histogram(ft_zq_Mass_nTpc_triggered[2], bins=b, range=r)
bin_centres = (bin_edges[:-1] + bin_edges[1:])/2.
errs = np.sqrt(counts)
# ax.errorbar(bin_centres, counts, yerr=errs, fmt='.', color='red', label=r"orig plot")

val=(r[1]-r[0])*1000 // b
ax.set_xlabel('$Mass, GeV$')
ax.set_ylabel(f'#events / {val}MeV')
ax.legend()
# fig

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

No handles with labels found to put in legend.


Let's normalize this plot:

In [13]:
plt.style.use(hep.style.ROOT)
fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot()

fig.suptitle('mass of events', fontsize=32)
b = 100
r = 0.7,2.7

n = 2
cnts = ax.hist(ft_zq_Mass_nTpc_triggered[n],bins=b, range=r,histtype='step', linewidth=2, density=True, stacked=True, label='correct trig', color='green')

ax.hist(ft_zq_Mass_nTpc_UNtriggered[n],bins=b, range=r,histtype='step', linewidth=2, density=True, label='fake trig', color='red',stacked=True)

val=(r[1]-r[0])*1000 // b
ax.set_xlabel('$Mass, GeV$')
ax.set_ylabel(f'#events / {val}MeV')

ax.legend()
# fig

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [609]:
%matplotlib widget

fig, ax = plt.subplots(1,3, figsize=(20,7))
fig.suptitle(r'zdc cut for mass of 4 track events')

b = 25
r = 0.5,3
counts,bin_edges = np.histogram(ft_zq_Mass_nTpc_triggered[2], bins=b, range=r)
# counts,bin_edges = np.histogram(kinematics.pt_events(ft.GetTracksWithNTPC(ft.four_tracks_zq)), bins=b, range=r)

bin_centres = (bin_edges[:-1] + bin_edges[1:])/2.
errs = np.sqrt(counts)
ax[0].errorbar(bin_centres, counts, yerr=errs, fmt='.', color='green')#, label=r'before zdc cut')


counts,bin_edges = np.histogram(ft_zq_Mass_nTpc_triggered[2][ind.index][ind], bins=b, range=r)
bin_centres = (bin_edges[:-1] + bin_edges[1:])/2.
errs = np.sqrt(counts)

ax[1].errorbar(bin_centres, counts, yerr=errs, fmt='.', color='black')# , label=r'zdc cut values')


counts,bin_edges = np.histogram(ft_zq_Mass_nTpc_triggered[2], bins=b, range=r)
bin_centres = (bin_edges[:-1] + bin_edges[1:])/2.
errs = np.sqrt(counts)

ax[2].errorbar(bin_centres, counts, yerr=errs, fmt='.', color='red') #, label=r'before zdc cut')

counts,bin_edges = np.histogram(ft_zq_Mass_nTpc_triggered[2][ind.index][~ind], bins=b, range=r)
# counts,bin_edges = np.histogram(kinematics.pt_events(ft.GetTracksWithNTPC(ft.four_tracks_zq)), bins=b, range=r)

bin_centres = (bin_edges[:-1] + bin_edges[1:])/2.
errs = np.sqrt(counts)
ax[2].errorbar(bin_centres, counts, yerr=errs, fmt='.', color='green') #, label=r'after zdc cut')



ax[0].legend()
ax[1].legend()
ax[2].legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.


### Pions subsystems

In our process 4 pions were producted. The most probably intermediate state including two pions and $\rho$ i.e.
$$\rho' \rightarrow \rho \ \pi^+ \pi^- \rightarrow \pi^+ \pi^- \pi^+ \pi^-$$ 

We can see this on distribution of mass that can be obtained as all combinations of pairs from intial four tracks, i.e. only four pairs:

![img](https://camo.githubusercontent.com/53a52e2a6d4ae7112d74f4073c979a51166170d8/68747470733a2f2f70702e757365726170692e636f6d2f633835333632342f763835333632343436372f34383466332f5431375a754b597062526f2e6a7067)

Here we can plot two distirbutions:

1. Make all possible(4) combinations of pairs. Then take lightest and pair that belong to one combination with that. Plot masses of these two pairs.
2. Plot masses of masses from possible combinations.

In [589]:
import modules.physics.analysis.pairs as pairs

tracks  = ft.GetTracksWithPtLt(ft.ft_zq_nTPC[0])
PosFirstTrack = tracks[tracks.T_Q > 0].groupby('entry').first()
PosSecTrack = tracks[tracks.T_Q > 0].groupby('entry').last()
NegFirstTrack = tracks[tracks.T_Q < 0].groupby('entry').first()
NegSecTrack = tracks[tracks.T_Q < 0].groupby('entry').last()

## first comb
pair1 = pairs.MakePair(PosFirstTrack, NegFirstTrack)
pair2 = pairs.MakePair(PosSecTrack, NegSecTrack)
## second comb
pair3 = pairs.MakePair(PosFirstTrack, NegSecTrack)
pair4 = pairs.MakePair(PosSecTrack, NegFirstTrack)

# first comb
m1 = kinematics.mass_events(pair1)
m2 = kinematics.mass_events(pair2)
## second comb
m3 = kinematics.mass_events(pair3)
m4 = kinematics.mass_events(pair4)


heavy_pairs1 = pd.concat([pair1.loc[m1[m1 > m2].index], pair2.loc[m2[m2 > m1].index]])
soft_pairs1 = pd.concat([pair1.loc[m1[m1 < m2].index], pair2.loc[m2[m2 < m1].index]])

heavy_pairs2 = pd.concat([pair3.loc[m3[m3 > m4].index], pair4.loc[m4[m4 > m3].index]])
soft_pairs2 = pd.concat([pair3.loc[m3[m3 < m4].index], pair4.loc[m4[m4 < m3].index]])



#  = pd.concat([heavy_pairs1, heavy_pairs2])

heavy_pairs = heavy_pairs1
heavy_pairs
# heavy_pairs = heavy_pairs2

# M = kinematics.mass_events(heavy_pairs)
# E = kinematics.E_events(heavy_pairs)
# P = kinematics.P_events(heavy_pairs)
# P0 = np.sqrt(M**2-4*(0.001*PiPlus.mass)**2)/2
# E0 = M / 2
# # cos_th1 = kinematics.cos_theta_events(heavy_pairs1[heavy_pairs1.T_Q > 0].groupby('entry').first(),heavy_pairs1)
# # tan_th1 = np.sqrt(1/cos_th**2 - 1)

# # theta_as = np.arcsin(P*E0*tan_th1/(E*P0*np.sqrt(1+tan_th1*E/M)))
# # theta_ac = np.arccos(1/(np.sqrt(1+1+tan_th1*E/M)))
# # theta = theta_as - theta_ac

# _ = plt.hist(M, bins=100, range=(0.3,2),histtype='step', linewidth=2, color='blue')


from modules.physics.analysis import pairs

# LiteHeavyRecoil, LiteHeavyTotal = pairs.GetPairs(ft.GetTracksWithPtLt(ft.GetTracksWithNTPC(ft.four_tracks_zq,4)))
LiteHeavyRecoil, LiteHeavyTotal = pairs.GetPairsMasses(ft.GetTracksWithPtLt(ft.GetTracksWithNTPC(ft.four_tracks_zq,4)))

fig = pairs.ShowMassComaprison(LiteHeavyTotal[ind.index][~ind], r'$\rho^0 \rightarrow \pi^+\pi^-$   and    $\pi^+\pi^-$ masses')
# # fig

KeyError: "None of [Int64Index([     8,     15,     20,     73,     82,    140,    155,    164,\n               178,    191,\n            ...\n            106383, 106392, 106402, 106457, 106497, 106499, 106524, 106601,\n            106636, 106687],\n           dtype='int64', name='entry', length=2848)] are in the [columns]"

In [15]:
mh1 = kinematics.mass_events(heavy_pairs1)
ms1 = kinematics.mass_events(soft_pairs1)
mh2 = kinematics.mass_events(heavy_pairs2)
ms2 = kinematics.mass_events(soft_pairs2)
pd.merge(pd.concat([mh1, mh2]).sort_index(), pd.concat([ms1, ms2]).sort_index(), right_on='entry', left_on='entry')

ValueError: Cannot merge a Series without a name

As we can see above, for second case (all possible pairs) we've got stronger signal in comparison with light-recoil pair as it made in [STAR work](http://arxiv.org/abs/0912.0604v2). Let's build 2d distirbuition and marginals component separately:

In [16]:
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec


fig = plt.figure(figsize=(15, 7))

gs = GridSpec(4,4,wspace=0.1,hspace=0.1)

ax_joint = fig.add_subplot(gs[1:4,0:3])
ax_marg_x = fig.add_subplot(gs[0,0:3])
ax_marg_y = fig.add_subplot(gs[1:4,3])

ax_joint.hist2d(LiteHeavyTotal.Recoil, LiteHeavyTotal.Lite, bins=(50, 50), range=[(0, 2), (0, 2)], cmap=plt.cm.jet)
_ = ax_marg_y.hist(LiteHeavyTotal.Lite, bins=100, range=(0, 2), histtype='step', color='blue', linewidth=2, label='lite pair',orientation="horizontal")
_ = ax_marg_x.hist(LiteHeavyTotal.Recoil, bins=100, range=(0, 2), histtype='step', color='red', linewidth=2, label='rest pair')


ax_joint.set_ylabel('Lightest pair Mass, GeV')
ax_joint.set_xlabel('Recoiling pair Mass, GeV')

# ax_marg_y.set_xlabel('$Mass, GeV$')
ax_marg_y.set_xlabel('# events')
ax_marg_x.yaxis.set_label_position("right")
ax_marg_x.xaxis.set_ticks([])
ax_marg_x.xaxis.set_ticks_position('none')
ax_marg_y.yaxis.set_ticks_position('none')
ax_marg_y.yaxis.set_ticks([])

# ax_marg_x.set_xlabel('$Mass, GeV$')
ax_marg_x.set_ylabel('# events')
ax_marg_y.xaxis.set_label_position("top")
ax_marg_x
ax_joint.legend()
# fig

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

No handles with labels found to put in legend.


## Decay angle for $\rho_0$

Here we would like to study angle decay for $\rho_0 \rightarrow \pi^+\pi^-$ between the momentum of one of the pions in the rest frame of $\rho_0$ and original(lab frame) momentum of  $\rho_0$.

For the sake of simplicity first we will create new lab system with OZ axis directed along momentum of $\rho_0$ in lab frame.

The transition from original lab frame to the new one could be obtain by two rotation:

- First, around x axis:

Rotation matrix for this case:
$$
R_{x,\alpha}=
\Bigg(
\begin{matrix}
1 & 0 & 0\\
0 & \cos{\alpha} & -\sin{\alpha} \\
0 & \sin{\alpha} & \cos{\alpha}
\end{matrix}
\Bigg)
$$

Rotation angle for x axis:

$$
\begin{matrix}
\sin{\alpha} =  \frac{p_y}{\sqrt{p_y^2+p_z^2}}  \\
\cos{\alpha} =  \frac{p_z}{\sqrt{p_y^2+p_z^2}}  \\
\end{matrix}
$$

Now, momentum vector of $\rho_0$ in this system will looks like

$$
\boldsymbol{p'}=R_{x,\alpha}\boldsymbol{p}=
\Bigg(
\begin{matrix}
p_x\\
p_y\cos{\alpha} - p_z\sin{\alpha}\\
p_y\sin{\alpha} + p_z\cos{\alpha})
\end{matrix}
\Bigg)
$$


- Second, around y axis:

Rotation matrix for this case will looks like
$$
R = R_{y,-\beta}= 
\Bigg(
\begin{matrix}
\cos{\beta} & 0 & -\sin{\beta}\\
0 & 1 & 0 \\
\sin{\beta} & 0 & \cos{\beta}
\end{matrix}
\Bigg)
$$

We have to apply it for already rotated vector around x axis:


$$
\boldsymbol{p''} =R_{y,-\beta}\boldsymbol{p'} = 
\Bigg(
\begin{matrix}
\cos{\beta} & 0 & -\sin{\beta}\\
0 & 1 & 0 \\
\sin{\beta} & 0 & \cos{\beta}
\end{matrix}
\Bigg)
\Bigg(
\begin{matrix}
{p'}_x\\
{p'}_y\\
{p'}_z
\end{matrix}
\Bigg)
 = 
\Bigg(
\begin{matrix}
{p'}_x\cos{\beta}-{p'}_z\sin{\beta}\\
{p'}_y\\
{p'}_x\sin{\beta}+{p'}_z\cos{\beta}
\end{matrix}
\Bigg)
$$


Where rotation angle based on the new rotated vector $\boldsymbol{p'}$ coordinates:

$$
\begin{matrix}
\sin{\beta}  =  \frac{{p'}_x}{\sqrt{{p'_x}^2+{p'}_z^2}}  \\
\cos{\beta}  =  \frac{{p'}_z}{\sqrt{{p'_x}^2+{p'}_z^2}}  \\
\end{matrix}
$$

As a result final transition looks like:
_____________
$$
\boldsymbol{p''} =
\Bigg(
\begin{matrix}
{p'}_x\cos{\beta}-{p'}_z\sin{\beta}\\
{p'}_y\\
{p'}_x\sin{\beta}+{p'}_z\cos{\beta}
\end{matrix}
\Bigg)=
\Bigg(
\begin{matrix}
p_x\cos{\beta}-p_y\sin{\alpha}\sin{\beta}-p_z\cos{\alpha}\sin{\beta} \\
p_y\cos{\alpha}-p_z\sin{\alpha} \\
p_x\sin{\beta}+p_y\sin{\alpha}\cos{\beta}+p_z\cos{\alpha}\cos{\beta}
\end{matrix}
\Bigg)
$$
where rotation angles are:


$$
\begin{matrix}
\sin{\alpha} =  \frac{p_y}{\sqrt{p_y^2+p_z^2}}  \\
\cos{\alpha} =  \frac{p_z}{\sqrt{p_y^2+p_z^2}}  \\
\sin{\beta}  =  \frac{p_x}{\sqrt{p_x^2+(p_y\sin{\alpha}+p_z\cos{\alpha})^2}}  \\
\cos{\beta}  =  \frac{p_y\sin{\alpha}+p_z\cos{\alpha}}{\sqrt{p_x^2+(p_y\sin{\alpha}+p_z\cos{\alpha})^2}}  \\
\end{matrix}
$$


Now let's consider moving coordinate system with $\rho_0$ so that OZ axis direct along $\boldsymbol{p}_{\rho_0}$

We know components of original momentum of $\pi^+$ in the such system and now let's boost their via Lorentz Transormation:

$$
\begin{matrix}
\ {E'} =  \gamma E - \Gamma p_z \\
\ {p'}_x= p_x \\
\ {p'}_y= p_y \\
\ {p'}_z= \gamma p_z - \Gamma E \\
\end{matrix}
$$

where 
$$ \boldsymbol{\beta} = \frac{\boldsymbol{p}}{E} $$
$$ \gamma = \frac{E}{m}$$
$$ \Gamma = \gamma \beta = \frac{p}{m}$$


$$ \boldsymbol{\beta} = \frac{\boldsymbol{p}}{E} $$
$$ \gamma = \frac{1}{\sqrt{1-\frac{p^2}{E^2}}}$$

Now the searched angle can be obtain from scalar multiplication of $\pi^+$ momentum in the rest frame of $\rho_0$ and momentum of $\rho_0$ in the lab frame:

$$\cos{\theta}=\frac{\boldsymbol{{p'}_{\pi^+}}\boldsymbol{p_{\rho_0}}} {{p'}_{\pi^+}p_{\rho_0}}$$

In [60]:
%matplotlib widget

tracks = heavy_pairs1[['T_Px', 'T_Py','T_Pz', 'T_Q']].sort_index()
tracks_rho = tracks.groupby('entry').sum()
tracks_pi_plus = tracks[tracks.T_Q>0].reset_index().drop('level_1',axis=1).set_index('entry')
tracks_pi_min = tracks[tracks.T_Q<0].reset_index().drop('level_1',axis=1).set_index('entry')

RotateToVector(rho, fir)
RotateToVector(rho, sec)
RotateToVector(tracks_rho, tracks_pi_plus)
RotateToVector(tracks_rho, tracks_pi_min)
BoostToSystem(rho.E/(0.001*Rho0.mass),rho.l/(0.001*Rho0.mass), fir)
BoostToSystem(rho.E/(0.001*Rho0.mass),rho.l/(0.001*Rho0.mass), sec)
BoostToSystem(tracks_rho.E/(0.001*Rho0.mass),tracks_rho.l/(0.001*Rho0.mass), tracks_pi_min)
BoostToSystem(tracks_rho.E/(0.001*Rho0.mass),tracks_rho.l/(0.001*Rho0.mass), tracks_pi_plus)

cost_lrnz_plus = CosTheta(tracks_pi_plus[['prx', 'pry', 'brpz']],tracks_rho[['T_Px', 'T_Py', 'T_Pz']])
cost_lrnz_pvn = CosTheta(fir[['prx', 'pry', 'brpz']],rho[['T_Px', 'T_Py', 'T_Pz']])
cost_pvn = CosTheta(fir[['T_Px', 'T_Py', 'T_Pz']],rho[['T_Px', 'T_Py', 'T_Pz']])
cost = CosTheta(tracks_pi_plus[['T_Px', 'T_Py', 'T_Pz']],tracks_rho[['T_Px', 'T_Py', 'T_Pz']])


lp_b_r_pi_plus = np.sqrt(tracks_pi_plus.prx**2 + tracks_pi_plus.pry**2 + tracks_pi_plus.brpz**2)

tracks_pi_plus['cos_theta'] = (tracks_pi_plus.prx*tracks_rho.T_Px + tracks_pi_plus.pry*tracks_rho.T_Py + tracks_pi_plus.brpz*tracks_rho.T_Pz) / (lp_b_r_pi_plus*tracks_rho.l)

_ = plt.hist(tracks_pi_plus.cos_theta,bins=100,range=(-1,1),histtype='step', linewidth=2, color='red', label=r"$\cos{{\theta}}$")

tracks_pi_plus['cost_orig'] = (tracks_pi_plus.T_Px*tracks_rho.T_Px + tracks_pi_plus.T_Py*tracks_rho.T_Py + tracks_pi_plus.T_Pz*tracks_rho.T_Pz) / (tracks_pi_plus.lpr * tracks_rho.l)

_ = plt.hist(tracks_pi_plus.cost_orig,bins=100,range=(-1,1),histtype='step', linewidth=2, color='black', label=r"$\cos{{\theta'}}$")
_ = plt.legend()


NameError: name 'RotateToVector' is not defined

## Cross section

Let's see to cross section of my events.

For this we should take luminosity of runs.

> Unfortunately file that I have to use for getting luminosity have a reference to special class AliTriggerInfo and moreover it packed into TObjArray, so I can't read it via uproot4. This is the reason why I used pure root again. [Here is the script](https://github.com/bdrum/cern-physics/blob/master/root-cpp/RhoPrime/macro/lumi.C) that I used.

Then let's see how much events do we have in each run.

$$L = \frac{1}{\sigma} \frac{\delta N}{\delta t}$$, this means that 

$$\sigma \approx \frac{N}{L}$$

Cross section of phenomena should be flat and independent from runs. 

Let's check it:

In [18]:
df_cs = GetCrossSection(ft.orig_events.RunNum, ft.GetTracksWithPtLt(ft.ft_zq_nTPC[0]))
df_cs['sigma'] = df_cs.nEvFT / (df_cs.Lumi * 1000)
df_cs.sigma.describe()


count    119.000000
mean       5.611438
std        1.558305
min        1.621205
25%        4.630173
50%        5.584997
75%        6.479376
max       10.036004
Name: sigma, dtype: float64

In [19]:
import scipy.stats

fig = plt.figure(figsize=(25, 7))
# fig.suptitle("visible cross section")
ax = fig.add_subplot()
bins = list(df_cs.run)
cnts = list(df_cs.sigma)
errs = np.sqrt(df_cs.nEvFT)/(df_cs.Lumi * 1000)
x = np.arange(len(bins))
width = 0.15
ax.errorbar(x - width/2, cnts, yerr=errs, fmt='o', label=r'$\sigma \approx \frac{N_{(\pi^+\pi^-\pi^+\pi^-)_{ev}}}{L}$')
ax.set_xticks(x)
ax.set_xticklabels(bins, rotation=90, rotation_mode="anchor", ha="right", fontsize=12)
ax.set_ylabel(r'$\sigma$',fontsize=14)
ax.set_ylim(0,10)

chi2,p = scipy.stats.chisquare(cnts, np.tile(np.average(cnts),len(cnts)))
ax.text(108,8,r"$\frac{Chi^2}{NDf} = \frac{189.388}{118}$", size=20)
ax.text(108,7,r"$p0 = 4.4542 \pm 0.11613$", size=16)
ax.legend(prop={'size': 15})
ax.axhline(y=np.average(cnts), color='r', linestyle='-')
ax.legend()
# fig

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Other decays

In PDG I've seen also other interesting modes for $\rho'$:

- 1. $\rho' \rightarrow \eta_0 \rho_0$ | ?
   - 1.1. $\rho_0 \rightarrow 4 \pi$ | $2*10^{-5}\%$
   - 1.2. $\rho_0 \rightarrow \pi^+ \pi^-$ | $10^{-2}\%$
   - 1.3. $\eta_0' \rightarrow  \pi^+ \pi^- \gamma$ | $4\%$
   - 1.4. $\eta_0' \rightarrow  \pi^+ \pi^- \pi^0$ | $23\%$
- 2. $\rho' \rightarrow 4 \pi$ | ?

What about $\rho' \rightarrow \rho_0 \rho_0$ is it possible?

## ZDC Energy distribution

Zero degree calorimeter (ZDC) has two part (A and C) aims to measure energy from neutrons that moves along pipe. [Such calorimiter has a purpose to measure collider luminosity based on elemctromagnetic dissotiation process](https://arxiv.org/pdf/1203.2436.pdf) that occures by ultra-peripheral collisions. This means that correct picture from ZDC aproves that we have UPC process.

In [176]:
%matplotlib widget

ft.orig_events.index.name = 'entry'

nTPC = 2
ind_times_mass_a = ft.orig_events[(np.abs(ft.orig_events[['ZDCAtime_0','ZDCAtime_1','ZDCAtime_2','ZDCAtime_3']])<t).apply(np.any,axis=1)].index.intersection(ft_zq_Mass_nTpc_triggered[nTPC].index)
ind_times_mass_c = ft.orig_events[(np.abs(ft.orig_events[['ZDCCtime_0','ZDCCtime_1','ZDCCtime_2','ZDCCtime_3']])<t).apply(np.any,axis=1)].index.intersection(ft_zq_Mass_nTpc_triggered[nTPC].index)

zdcEn_A = ft.orig_events.ZNAenergy[ind_times_mass_a]
zdcEn_C = ft.orig_events.ZNCenergy[ind_times_mass_c]
zdcEn = np.concatenate((zdcEn_A,zdcEn_C))
# zdcEn = np.concatenate((ft.orig_events.ZNAenergy[ft_zq_Mass_nTpc_triggered[nTPC].index], ft.orig_events.ZNCenergy[ft_zq_Mass_nTpc_triggered[nTPC].index]))
# zdcEn = np.concatenate((ft.orig_events.ZNAenergy[ft_nzq_Pt_nTpc[nTPC].index], ft.orig_events.ZNCenergy[ft_nzq_Pt_nTpc[nTPC].index]))
cnt = plt.hist(zdcEn, bins=100, range=(-1000,15000), histtype='step', linewidth=1.3, color='black')
plt.xlabel('ZDC energy')
plt.ylabel('events')
# plt.yscale('log')
plt.suptitle('4track events ZDC energy distribution')
plt.legend()

ds = { \
    'count': len(zdcEn), \
    # 'mean' : np.average(zdcEn), \
    # 'std'  : np.std(zdcEn), \
    # 'min'  : zdcEn.min(), \
    # 'max'  : zdcEn.max(), \
    # '% < 0 TeV' : 100 * (zdcEn <= 0).sum() / len(zdcEn), \
    # '% < 2.5TeV' : 100 * (zdcEn <= 2500).sum() / len(zdcEn), \
    # '% < 5TeV' : 100 * (zdcEn <= 5000).sum() / len(zdcEn), \
    # '% < 7.5TeV' : 100 * (zdcEn <= 7500).sum() / len(zdcEn), \
    # '% < 10TeV' : 100 * (zdcEn <= 10000).sum() / len(zdcEn) ,\
     '>= 15TeV' : (zdcEn >= 15000).sum() \
    }

ds = pd.DataFrame.from_dict(ds, orient='index').round(1)

plt.axes([0.4, 0.4, 0.8, 0.8],frame_on=False)
plt.xticks([])
plt.yticks([])

stats = plt.table(cellText=ds.values,
      colWidths = [0.2]*2,
      rowLabels=ds.index,
      cellLoc = 'right', rowLoc = 'left',
      loc=(10,0))

stats.set_fontsize(16)
stats.scale(1,3)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

No handles with labels found to put in legend.


In [175]:
%matplotlib widget


zdcTimes_C = np.concatenate((ft.orig_events.ZDCCtime_0[ft.orig_events.ZNCenergy>1000], ft.orig_events.ZDCCtime_1[ft.orig_events.ZNCenergy>1000],ft.orig_events.ZDCCtime_2[ft.orig_events.ZNCenergy>1000],ft.orig_events.ZDCCtime_3[ft.orig_events.ZNCenergy>1000]))

# zdcTimes_A = np.concatenate((ft.orig_events.ZDCAtime_0[ft_zq_Mass_nTpc_triggered[nTPC].index], ft.orig_events.ZDCAtime_1[ft_zq_Mass_nTpc_triggered[nTPC].index], ft.orig_events.ZDCAtime_2[ft_zq_Mass_nTpc_triggered[nTPC].index], ft.orig_events.ZDCAtime_3[ft_zq_Mass_nTpc_triggered[nTPC].index]))

zdcTimes_A = np.concatenate((ft.orig_events.ZDCAtime_0[ft.orig_events.ZNAenergy>1000], ft.orig_events.ZDCAtime_1[ft.orig_events.ZNAenergy>1000], ft.orig_events.ZDCAtime_2[ft.orig_events.ZNAenergy>1000], ft.orig_events.ZDCAtime_3[ft.orig_events.ZNAenergy>1000]))



ft.orig_events.ZNAenergy

plt.subplots(1,2, figsize=(20,10))
plt.suptitle('4track events ZDC time distribution')
plt.sca(plt.gcf().axes[0])

cnt = plt.hist(zdcTimes_A, bins=100, range=(-300,300), histtype='step', linewidth=1.3, color='black')
plt.xlabel('ZDC_A time (ns)')
plt.ylabel('events')
plt.yscale('log')
plt.legend()

plt.sca(plt.gcf().axes[1])

cnt = plt.hist(zdcTimes_C, bins=100, range=(-300,300), histtype='step', linewidth=1.3, color='black')
plt.xlabel('ZDC_C time (ns)')
plt.ylabel('events')
plt.yscale('log')
plt.suptitle('4track events ZDC time distribution')
plt.legend()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

No handles with labels found to put in legend.
No handles with labels found to put in legend.


## ZDC Times

- First plot ZDC energy for candidates with time around zero
- Second plot ZDC energy with itme more than zero


In [169]:
# np.concatenate((ft.orig_events.ZDCAtime_0[ft.orig_events.ZNAenergy>1000], ft.orig_events.ZDCAtime_1[ft.orig_events.ZNAenergy>1000], ft.orig_events.ZDCAtime_2[ft.orig_events.ZNAenergy>1000], ft.orig_events.ZDCAtime_3[ft.orig_events.ZNAenergy>1000]))[:10]

t = 10
# ft.orig_events[['ZNAenergy', 'ZDCAtime_0','ZDCAtime_1','ZDCAtime_2','ZDCAtime_3']][ft.orig_events.ZNAenergy>1000][]


# ft.orig_events[(np.abs(ft.orig_events[['ZDCAtime_0','ZDCAtime_1','ZDCAtime_2','ZDCAtime_3']])<t).apply(np.any,axis=1)].ZNAenergy


entry
20        35752.679688
155        3246.793945
164       20386.974609
178        5147.336426
242       34955.371094
              ...     
109123    44199.542969
109277    41881.656250
109302    21917.554688
109394     3232.702881
109698    21158.830078
Name: ZNAenergy, Length: 496, dtype: float32

In [165]:
df1 = pd.DataFrame(np.arange(0,9).reshape(3,3), columns=list('abc'))

c = pd.Series([0,2,3,4,5,6])
# df1[c]
df1


,a,b,c
0,0,1,2
1,3,4,5
2,6,7,8


In [116]:
list(')

['1', '2', '3']

In [75]:
%matplotlib widget

nTPC = 2
zdcEnA = ft.orig_events.ZNAenergy[ft_zq_Mass_nTpc_triggered[nTPC].index]
zdcEnC = ft.orig_events.ZNCenergy[ft_zq_Mass_nTpc_triggered[nTPC].index]

# _ = plt.hist2d(zdcEnA,zdcEnC, bins=(
        # 100, 100), range=[(-1000,15000), (-1000,15000)], cmap=plt.cm.Greys)
_ = plt.scatter(zdcEnA, zdcEnC, color='black')
plt.xlabel('ZDC_A energy')
plt.ylabel('ZDC_C energy')
plt.xlim(-1000, 11000)
plt.ylim(-1000, 11000)
plt.xticks(np.arange(0,11000,2500))
plt.yticks(np.arange(0,11000,2500))
plt.suptitle('4track events ZDC_A vs ZDC_C scatter')
plt.legend()

# zdcEn = np.concatenate((ft.orig_events.ZNAenergy[ft_nzq_Mass_nTpc[nTPC].index], ft.orig_events.ZNCenergy[ft_nzq_Mass_nTpc[nTPC].index]))

# _ =plt.hist(zdcEn, bins=100, range=(-1000,15000), histtype='step', linewidth=1.3, color='red')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

No handles with labels found to put in legend.


In [77]:
%matplotlib widget

nTPC = 2
zdcTimes_C = np.concatenate((ft.orig_events.ZDCCtime_0[ft_zq_Mass_nTpc_triggered[nTPC].index], ft.orig_events.ZDCCtime_1[ft_zq_Mass_nTpc_triggered[nTPC].index], ft.orig_events.ZDCCtime_2[ft_zq_Mass_nTpc_triggered[nTPC].index], ft.orig_events.ZDCCtime_3[ft_zq_Mass_nTpc_triggered[nTPC].index]))

zdcTimes_A = np.concatenate((ft.orig_events.ZDCAtime_0[ft_zq_Mass_nTpc_triggered[nTPC].index], ft.orig_events.ZDCAtime_1[ft_zq_Mass_nTpc_triggered[nTPC].index], ft.orig_events.ZDCAtime_2[ft_zq_Mass_nTpc_triggered[nTPC].index], ft.orig_events.ZDCAtime_3[ft_zq_Mass_nTpc_triggered[nTPC].index]))

# _ = plt.hist2d(zdcEnA,zdcEnC, bins=(
        # 100, 100), range=[(-1000,15000), (-1000,15000)], cmap=plt.cm.Greys)
_ = plt.scatter(zdcTimes_A, zdcTimes_C, color='black')
plt.xlabel('ZDC_A energy')
plt.ylabel('ZDC_C energy')
plt.xlim(-300, 300)
plt.ylim(-300, 300)
# plt.xticks(np.arange(0,11000,2500))
# plt.yticks(np.arange(0,11000,2500))
plt.suptitle('4track events ZDC_A vs ZDC_C scatter')
plt.legend()

# zdcEn = np.concatenate((ft.orig_events.ZNAenergy[ft_nzq_Mass_nTpc[nTPC].index], ft.orig_events.ZNCenergy[ft_nzq_Mass_nTpc[nTPC].index]))

# _ =plt.hist(zdcEn, bins=100, range=(-1000,15000), histtype='step', linewidth=1.3, color='red')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

No handles with labels found to put in legend.


In [620]:
%matplotlib widget 
nTPC=2
pt_with_cut = kinematics.pt_events(ft.GetTracksWithPtLt(ft.ft_zq_nTPC[nTPC], maxPt=2))
pt_with_cut = ft_zq_Pt_nTpc_triggered[2]
v1 = 3000
v2 = np.inf
ind = (pd.concat([ft.orig_events.ZNAenergy[pt_with_cut.index], ft.orig_events.ZNCenergy[pt_with_cut.index]]) > v1).sort_index().groupby('entry').sum() >= 1
ind *= (pd.concat([ft.orig_events.ZNAenergy[pt_with_cut.index], ft.orig_events.ZNCenergy[pt_with_cut.index]]) <= v2).sort_index().groupby('entry').sum() >= 1
_ = plt.hist(pt_with_cut[ind.index][~ind], bins=100, range=(0, 2), histtype='step', color='black', linewidth=1.2, label='ZDC enrgy <= 3 TeV')
_ = plt.hist(pt_with_cut[ind.index][ind], bins=100, range=(0, 2), histtype='step', color='red', linewidth=1.2, label='ZDC enrgy > 3 TeV')
plt.xlabel(r'$p_t$')
plt.ylabel('events')
# plt.suptitle(f'4tracks event $p_t$ when {v1/1000}TeV < ZDC energy <= {v2}')
_ = plt.legend()

C:\Python38\lib\site-packages\pandas\core\computation\expressions.py:204: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  warnings.warn(


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## $ \downarrow \downarrow \downarrow \downarrow$ DEBUG AND INVESTIGATIONS $ \downarrow \downarrow \downarrow \downarrow$


In [603]:
%matplotlib widget 
nTPC=2
mass = ft_zq_Mass_nTpc_triggered[nTPC]
v1 = 3000
v2 = np.inf
ind = (pd.concat([ft.orig_events.ZNAenergy[mass.index], ft.orig_events.ZNCenergy[mass.index]]) > v1).sort_index().groupby('entry').sum() >= 1
ind *= (pd.concat([ft.orig_events.ZNAenergy[mass.index], ft.orig_events.ZNCenergy[mass.index]]) <= v2).sort_index().groupby('entry').sum() >= 1
_ = plt.hist(mass[ind.index][~ind], bins=100, range=(0.5, 3), histtype='step', color='black', linewidth=1.2, label='ZDC enrgy <= 3 TeV')
# _ = plt.hist(mass[ind.index][ind], bins=25, range=(0.5, 3), histtype='step', color='red', linewidth=1.2, label='ZDC enrgy > 3 TeV')
plt.xlabel(r'$p_t$')
plt.ylabel('events')
# plt.suptitle(f'4tracks event $p_t$ when {v1/1000}TeV < ZDC energy <= {v2}')
_ = plt.legend()

C:\Python38\lib\site-packages\pandas\core\computation\expressions.py:204: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  warnings.warn(


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Make pair of tracks more comprehensive 

In [591]:
# ft.orig_events.ZNAenergy
ft.orig_events[]

Index(['RunNum', 'PeriodNumber', 'OrbitNumber', 'BunchCrossNumber', 'Mass',
       'Pt', 'Q', 'Rapidity', 'Phi', 'ZNAenergy', 'ZNCenergy', 'ZPAenergy',
       'ZPCenergy', 'VtxX', 'VtxY', 'VtxZ', 'VtxContrib', 'VtxChi2', 'VtxNDF',
       'SpdVtxX', 'SpdVtxY', 'SpdVtxZ', 'SpdVtxContrib', 'V0Adecision',
       'V0Cdecision', 'ADAdecision', 'ADCdecision', 'V0Afired', 'V0Cfired',
       'ADAfired', 'ADCfired', 'STPfired', 'SMBfired', 'SM2fired', 'SH1fired',
       'OM2fired', 'OMUfired', 'IsTriggered', 'nTracklets', 'nTracks',
       'FORChip'],
      dtype='object')

In [58]:
# pvn_df = pd.read_csv(r'D:/GoogleDrive/Job/cern/Alice/analysis/data/log.csv', delimiter='\t')
# pvn_df = pvn_df.rename(columns={'px': 'T_Px', 'py' : 'T_Py', 'pz' : 'T_Pz'})
# pvn_df = pvn_df.apply(pd.to_numeric)
fir = pvn_df[::2].reset_index()
sec = pvn_df[1::2].reset_index()
rho = fir + sec

rho['l'] = np.sqrt(rho.T_Px**2+rho.T_Py**2+rho.T_Pz**2)
rho['E'] = np.sqrt(rho.T_Px**2 + rho.T_Py**2 + rho.T_Pz**2 + (0.001*Rho0.mass)**2)
p = (fir + sec).T_Px**2 + (fir + sec).T_Py**2 + (fir + sec).T_Pz**2
e1 = np.sqrt(fir.T_Px**2 + fir.T_Py**2 + fir.T_Pz**2 + (0.001*PiPlus.mass)**2)
e2 = np.sqrt(sec.T_Px**2 + sec.T_Py**2 + sec.T_Pz**2 + (0.001*PiPlus.mass)**2)
e = e1 + e2
m = np.sqrt(e**2 - p)

_ = plt.hist(m, bins=100,range=(0.3,1),histtype='step', linewidth=2, color='black', label=r"m_{\rho}$")


NameError: name 'pvn_df' is not defined

In [37]:
df = heavy_pairs1[['T_Px', 'T_Py','T_Pz', 'T_Q']].sort_index()
with open(r'D:\GoogleDrive\Job\cern\Alice\analysis\data\4pi_rho.csv', 'w') as f:
    f.write('T_Px\tT_Py\tT_Pz\tT_Q\n')
    for i in range(len(df)):
        f.write(f'{df.iloc[i].T_Px}\t{df.iloc[i].T_Py}\t{df.iloc[i].T_Pz}\t{df.iloc[i].T_Q}\n')
        if (i % 2 != 0):
            f.write('\n')


In [42]:
runs = ft.orig_events[['RunNum']].loc[ft_zq_Mass_nTpc_triggered[2].index]
runs['cnt'] = 1
runs = runs.groupby('RunNum').sum().sort_values(by='cnt', ascending=False)
runs
cd $__annotations__
# runs.to_csv(r"D:\GoogleDrive\Job\cern\Alice\analysis\data\RhoPrime\2015\run_ev.csv", sep=' ')

,cnt
RunNum,
246087,125
246989,106
246980,72
245554,71
245507,70
...,...
246808,2
245231,2
246759,2
